# Marijuana Anonymous Cravings Support RAG Pipeline v2.0

Builds on v1.0 but puts vector & hybrid search in place of elasticsearch in RAG system

In [17]:
# load utility functions
import json
from pathlib import Path
from dotenv import load_dotenv
import os
load_dotenv()
from tqdm.auto import tqdm
from datetime import datetime

In [2]:
# Load Open AI API client 
from openai import OpenAI
openai_client = OpenAI(api_key=os.getenv("OPENAI_API_KEY")) # Get Key from Env (ignored in git) 
# client = OpenAI()

## 1. Load User Data

Load user information from `users.json` and prompt for user ID.

In [8]:
# Path to users.json
users_path = Path("../context/users/users.json")

In [9]:
# initialize current conversation history
current_conversation_history = []

In [10]:
def load_users():
    if users_path.exists():
        with open(users_path, "r") as f:
            return json.load(f)
    return []

In [11]:
def get_user_by_id(user_id, users):
    for user in users:
        if user.get("user_id") == user_id:
            return user
    return None

In [12]:
#get user id
user_id = input("Enter your user_id: ").strip()

Enter your user_id:  00006


In [14]:
# get user info
users = load_users() 
user = get_user_by_id(user_id, users)
if user: 
    print(f"User found: {user}")
else:
    print(f"No user found with user_id: {user_id}")
    exit(1)

User found: {'user_id': '00006', 'user_name': 'SarahWilson', 'first_name': 'Sarah', 'last_name': 'Wilson', 'created_at': '2025-06-11T20:53:48.928579', 'persona': 'steady_believer', 'view': 'christian', 'step': None, 'step_in_progress': False, 'first_sober_date': '2025-06-01', 'openness_level': 'medium', 'last_updated': '2025-06-13T09:01:31.701906'}


## 2. Load Spiritual Profile

Load the user's spiritual profile from `spiritual_profile.json`.

In [15]:
# Load spiritual profile
spiritual_profile_path = Path("../context/spiritual_profile/spiritual_profile.json")
if spiritual_profile_path.exists():
    with open(spiritual_profile_path, "r") as f:
        profiles = json.load(f)
    user_profile = next((p for p in profiles if p["user_id"] == user_id), None)
else:
    user_profile = None

## 3. Load Sponsor Persona Info
## 4. Store user, spiritual profile, and persona in Variables

In [18]:
# store user info to variables. 
user_id = user['user_id']
first_name = user.get('first_name', 'Unknown')
last_name = user.get('last_name', 'Unknown')
view = user.get('view', 'Unknown')
persona = user.get('persona', 'Unknown')
step = user.get('step', 'Unknown')
step_in_progress = user.get('step_in_progress', 'Unknown')
first_sober_date = user.get("first_sober_date")
if first_sober_date:
    sober_days = (datetime.now() - datetime.strptime(first_sober_date, "%Y-%m-%d")).days
else:
    sober_days = None

# store spiritual profile info to variables.
openness_level = user_profile['current_profile']['openness_level']
higher_power_defined = user_profile['current_profile']['higher_power_defined']

# Store sponsor persona info to variables.
persona_path = Path("../context/personas/personas.json")
if persona_path.exists():
    with open(persona_path, "r") as f:
        personas = json.load(f)
    persona_info = next((p for p in personas if p["persona_id"] == persona), None)
else:
    persona_info = None

persona_name = persona_info.get('name', 'No persona name available') if persona_info else 'No persona information available'
persona_description = persona_info.get('description', 'No description available') if persona_info else 'No persona information available'
perona_language_style = persona_info.get('language_style', 'No language style available') if persona_info else 'No persona information available'

In [19]:
print(user_id)
print(first_name)
print(last_name)
print(view)
print(persona)
print(step)
print(step_in_progress)
print(sober_days)
print(openness_level)
print(higher_power_defined)
print(persona_name)
print(persona_description)
print(perona_language_style)

00006
Sarah
Wilson
christian
steady_believer
None
False
31
medium
True
Steady Believer
Faith-driven, emotionally expressive, trusts spiritual authority
Warm, scriptural, relational


## 4. Load Previous Conversations and Summarize

Load previous conversations for the user and summarize recent activity.

In [21]:
# --- Load and summarize previous conversations ---

conversations_path = Path("../context/conversations/conversations.json")
recent_convos = []
recent_cravings_convos = []

if conversations_path.exists():
    with open(conversations_path, "r") as f:
        conversations = json.load(f)
    # Filter for this user
    user_convos = [c for c in conversations if c.get("user_id") == user_id]
    # Sort by timestamp (descending, most recent first)
    user_convos = sorted(user_convos, key=lambda x: x.get("timestamp", ""), reverse=True)
    # Last 3 overall conversations
    recent_convos = user_convos[:3]
    # Last 3 cravings conversations (if any)
    cravings_convos = [c for c in user_convos if c.get("conversation_type") == "cravings"]
    recent_cravings_convos = cravings_convos[:3]
else:
    print("No conversations found.")

def summarize_convos(convos):
    summaries = []
    for c in convos:
        summary = {
            "summary": c.get("conversation_data", {}).get("summary"),
            "key_insights": c.get("key_insights"),
            "action_items": c.get("action_items"),
            "last_dialogue": c.get("conversation_data", {}).get("dialogue", []),
            "emotional_tone_end": c.get("emotional_tone_end"),
            "conversation_type": c.get("conversation_type"),
            "timestamp": c.get("timestamp")
        }
        summaries.append(summary)
    return summaries

# Summaries for last 3 overall conversations
recent_convo_summaries = summarize_convos(recent_convos)
# Summaries for last 3 cravings conversations
recent_cravings_summaries = summarize_convos(recent_cravings_convos)

# Example: print summaries for review
print("Last 3 conversations:")
for i, convo in enumerate(recent_convo_summaries, 1):
    print(f"\nConversation {i}:")
    print(f"  Summary: {convo['summary']}")
    print(f"  Key Insights: {convo['key_insights']}")
    print(f"  Action Items: {convo['action_items']}")
    print(f"  Last Dialogue: {convo['last_dialogue']}")
    print(f"  Emotional Tone End: {convo['emotional_tone_end']}")
    print(f"  Conversation Type: {convo['conversation_type']}")
    print(f"  Timestamp: {convo['timestamp']}")

print("\nLast 3 cravings conversations:")
for i, convo in enumerate(recent_cravings_summaries, 1):
    print(f"\nCravings Conversation {i}:")
    print(f"  Summary: {convo['summary']}")
    print(f"  Key Insights: {convo['key_insights']}")
    print(f"  Action Items: {convo['action_items']}")
    print(f"  Last Dialogue: {convo['last_dialogue']}")
    print(f"  Emotional Tone End: {convo['emotional_tone_end']}")
    print(f"  Conversation Type: {convo['conversation_type']}")
    print(f"  Timestamp: {convo['timestamp']}")

Last 3 conversations:

Conversation 1:
  Summary: Sarah is exploring her relationship with marijuana and is open to discussing spirituality as part of his recovery journey. He feels stuck in achieving his life goals and is curious about how others manage similar situations.
  Key Insights: ['Sarah feels marijuana is making him lazy and unproductive.', 'He wants to buy a house and have a family but feels stuck.', 'Sarah is uncertain about being part of a community but is open to exploring spirituality.']
  Action Items: ['Explore how faith might support recovery.']
  Last Dialogue: [{'role': 'user', 'content': 'i just wanted to check things out'}, {'role': 'sponsor', 'content': "I'm really glad you decided to check things out today. It's a good first step, and everyone here has taken it at some point. What's your name, if you don’t mind sharing?"}, {'role': 'user', 'content': 'Sarah'}, {'role': 'sponsor', 'content': "Hi Sarah, it's really nice to meet you. I'm glad you're here. What bro

In [41]:
# Format summaries and key insights for the prompt
summaries_text = "\n".join(
    [f"- {c['summary']}" for c in recent_convo_summaries if c['summary']]
)
key_insights_text = "\n".join(
    [f"- {insight}" for c in recent_convo_summaries for insight in (c['key_insights'] or [])]
)

## 5. Format User, Persona, and Conversation Info for Prompt

Prepare user info, persona description, and recent conversation summaries for use in the LLM prompt.

In [62]:
# Format user info for the prompt
user_info_text = f"""
User Info:
- Name: {first_name} {last_name}
- Spiritual View: {view}
- Step: {step} (In Progress: {step_in_progress})
- Sober Days: {sober_days}
- Openness Level: {openness_level}
- Higher Power Defined: {higher_power_defined}
"""

# Format persona info for the prompt
persona_text = f"""
AI Sponsor Persona: {persona}
{persona_description}
"""

# Format recent conversation summaries and key insights
recent_summaries_text = "\n".join(
    [f"- {c['summary']}" for c in recent_convo_summaries if c['summary']]
)
recent_key_insights_text = "\n".join(
    [f"- {insight}" for c in recent_convo_summaries for insight in (c['key_insights'] or [])]
)

In [47]:
print(user_info_text)
print(persona_text)
print(recent_summaries_text)
print(recent_key_insights_text)


User Info:
- Name: Sarah Wilson
- Spiritual View: christian
- Step: None (In Progress: False)
- Sober Days: 31
- Openness Level: medium
- Higher Power Defined: True


AI Sponsor Persona: steady_believer
Faith-driven, emotionally expressive, trusts spiritual authority

- Sarah is exploring her relationship with marijuana and is open to discussing spirituality as part of his recovery journey. He feels stuck in achieving his life goals and is curious about how others manage similar situations.
- Sarah feels marijuana is making him lazy and unproductive.
- He wants to buy a house and have a family but feels stuck.
- Sarah is uncertain about being part of a community but is open to exploring spirituality.


### Load and Prepare the Corpus

In [102]:
# --- Doctors Opinion ---
doctors_path = Path("../context/doctors_opinion/doctors_opinion.json")

doc_opinion = json.load(open(doctors_path, "r"))
with open(doctors_path, "r", encoding="utf-8") as f:
    doc_opinion = json.load(f)

In [103]:
# Prepare list of chunks with metadata
docs_to_index = []

for section in doc_opinion.get("content_sections", []):
    docs_to_index.append({
        "id": section.get("id", ""),
        "source": doc_opinion["document_metadata"]["source"],
        "title": section.get("title", ""),
        "text": section.get("text", ""),
        "tags": section.get("tags", []),
        "conversation_types": section.get("conversation_types", []),
        "user_scenarios": section.get("user_scenarios", []),
    })

### Embed and Store in Qdrant

In [63]:
!python -m pip install -q "qdrant-client[fastembed]>=1.14.2"


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python -m pip install --upgrade pip


In [64]:
from qdrant_client import QdrantClient, models

In [65]:
qd_client = QdrantClient("http://localhost:6333") #connecting to local Qdrant instance

In [68]:
model_handle = "jinaai/jina-embeddings-v2-small-en"

In [69]:
# Initialize the embedding model
embedding_model = TextEmbedding(model_name=model_handle)

Fetching 5 files: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:01<00:00,  3.18it/s]


In [126]:
EMBEDDING_DIMENSIONALITY = 512
# Define the collection name
collection_name = "ma_cravings"

# Create the collection with specified vector parameters
qd_client.create_collection(
    collection_name=collection_name,
    vectors_config=models.VectorParams(
        size=EMBEDDING_DIMENSIONALITY,  # Dimensionality of the vectors
        distance=models.Distance.COSINE  # Distance metric for similarity search
    )
)

True

### 2d Create, embed, & Insert Points into the Collection 

In [128]:
from qdrant_client.models import PointStruct
import uuid

points = []

for doc in docs_to_index:
    text = doc["text"]
    vector = list(embedding_model.embed([text]))[0]  # embedding as 512-dim vector
    point = PointStruct(
        id=str(uuid.uuid4()),
        vector=vector,
        payload=doc  # includes all metadata: source, tags, title, user_scenarios, etc.
    )
    points.append(point)

qd_client.upsert(collection_name=collection_name, points=points)
print(f"✅ {len(points)} docs inserted into collection '{collection_name}'")


✅ 15 docs inserted into collection 'ma_cravings'


In [129]:
query = "I'm really craving right now and I feel like using"
vector = list(embedding_model.embed([query]))[0]


## 7. Retrieval Function

Define a function to retrieve relevant passages from Vector Search based on user queries.

In [140]:
from qdrant_client.models import Document

def semantic_search(query, client, collection_name, model_handle, limit=3):
    results = client.query_points(
        collection_name=collection_name,
        query=Document(
            text=query,
            model=model_handle  # e.g., "jinaai/jina-embeddings-v2-small-en"
        ),
        limit=limit,
        with_payload=True
    )
    return results


In [149]:
query = "I'm experiencing intense cravings to use again."

results = semantic_search(
    query=query,
    client=qd_client,
    collection_name="ma_cravings",
    model_handle=model_handle
)


In [148]:
for point in results.points:
    print(f"Score: {point.score:.3f}")
    print(f"Title: {point.payload.get('title')}")
    print(point.payload.get('text')[:300])
    print("-" * 60)

Score: 0.823
Title: Withdrawal and Internal Stress Systems
Regular, long-term use results in activation of internal stress systems which try to keep brain function normal. These stress systems cause adverse feelings such as anxiety and dysphoria resulting in a fairly negative emotional state which is felt during withdrawal. This negative emotional state bec
------------------------------------------------------------
Score: 0.823
Title: Understanding Cravings and Preoccupation
The third stage of addiction is related to craving. Craving can be described as preoccupation and anticipation of marijuana use. The frontal cortex, where we think things through, plan things out, and alter our behavior to meet our own needs, is the primary part of the brain that is altered. As a re
------------------------------------------------------------
Score: 0.822
Title: Tolerance and Using to Feel Normal
Another change to the brain during active addiction is related to negative feelings users begin to ha

In [158]:
def format_context(results):
    context = ""
    for result in results:
        context += f"Source: {result['source']}\nTitle: {result.get('title','')}\nText: {result['text']}\n\n"
    return context.strip()

In [160]:
def format_context(results):
    return "\n\n".join([f"{point.payload.get('title')}\n{point.payload.get('text')}" for point in results.points])


In [162]:
print(format_context(results))

Withdrawal and Internal Stress Systems
Regular, long-term use results in activation of internal stress systems which try to keep brain function normal. These stress systems cause adverse feelings such as anxiety and dysphoria resulting in a fairly negative emotional state which is felt during withdrawal. This negative emotional state becomes much longer lasting and is relieved only by a return to use of marijuana or other drugs, unless the addict stops using altogether and for a substantial period of time, allowing the entire system to return to normal function.

Understanding Cravings and Preoccupation
The third stage of addiction is related to craving. Craving can be described as preoccupation and anticipation of marijuana use. The frontal cortex, where we think things through, plan things out, and alter our behavior to meet our own needs, is the primary part of the brain that is altered. As a result of addiction, the frontal lobes are no longer functioning at full capacity, limiting

## 8. Build RAG Prompt

Construct the prompt for the LLM using user info, persona, conversation history, and retrieved context.

In [163]:
def build_rag_prompt(
    user_query, context, user_info_text, persona_text,
    recent_summaries_text, recent_key_insights_text, conversation_history
):
    # Convert dicts to readable strings
    history_text = "\n".join(
        f"{msg['role'].capitalize()}: {msg['content']}" for msg in conversation_history
    )
    prompt = f"""
You are a Marijuana Anonymous AI sponsor. Take on the persona described below.

MODULE: Cravings Support
Your primary goal is to help the user manage and overcome cravings for marijuana today.

{persona_text}

USER INFO:
{user_info_text}

RECENT CONVERSATION SUMMARIES:
{recent_summaries_text}

KEY INSIGHTS FROM RECENT CONVERSATIONS:
{recent_key_insights_text}

CONVERSATION SO FAR:
{history_text}

USER QUERY:
{user_query}

CONTEXT FROM MA LITERATURE:
{context}

Instructions:
- Respond in a warm, conversational tone.
- Limit your response to 2-3 sentences.
- Focus on cravings management, relapse prevention, and spiritual encouragement.
- Reference the context or literature if helpful.
- Always end with a supportive, open-ended question to keep the conversation going.
- If the context does not contain the answer, respond with empathy and general support, using the AI sponsor persona.
"""
    return prompt.strip()

## 9. Run the RAG Pipeline

Main loop: accept user queries, retrieve context, build prompt, get LLM response, and update conversation history.

In [167]:
while True: 
    user_query = input("Describe your craving or ask a question (or type 'quit' to exit): ")
    if user_query.strip().lower() in ["quit", "exit"]:
        print("Ending conversation. Take care!")
        break
    retrieved_docs = semantic_search(
        query=user_query,
        client=qd_client,
        collection_name="ma_cravings",
        model_handle=model_handle
    )

    context = format_context(retrieved_docs)
    prompt = build_rag_prompt(
        user_query, context, persona_text, user_info_text,
        recent_summaries_text, recent_key_insights_text, current_conversation_history
    )

    response = openai_client.chat.completions.create(
        model='gpt-4o',
        messages=[{"role": "user", "content": prompt}]
    )
    ai_response = response.choices[0].message.content
    print(ai_response)

    # Store as dicts for compatibility with extract_conversation_insights
    current_conversation_history.append({"role": "user", "content": user_query})
    current_conversation_history.append({"role": "sponsor", "content": ai_response})

Describe your craving or ask a question (or type 'quit' to exit):  i cant stop thinking about weed


Hi Sarah, I understand how strong those cravings can feel, especially when they keep creeping into your thoughts. It's a tough battle when you're dealing with both the cravings and the feeling of being stuck. Remember, you’re not alone, and talking to your Higher Power can be a source of strength right now. What are some spiritual practices or thoughts you find comforting when you feel overwhelmed?


Describe your craving or ask a question (or type 'quit' to exit):  its like im trying to do work, but all i think about is going to the cannabis store


It's really challenging when those thoughts start coming back, especially when you're trying to focus on work and your goals. Remember, it's okay to acknowledge the craving and then gently redirect your attention to something that inspires or uplifts you. Maybe take a moment to connect with your Higher Power and ask for strength and clarity in this moment. How do you feel about finding a quiet space for a quick prayer or meditation to help refocus your mind?


Describe your craving or ask a question (or type 'quit' to exit):  well im trying to get work done and only have an hour till the end of the day. i dont think i have time to meditate


...

It sounds like a peaceful ending to your day, Sarah, and praying when you feel calm can strengthen your connection to your Higher Power. Love and spirituality are seen as rewarding experiences, aiding your journey to heal and find hope. How do you plan to incorporate these feelings of love and spirituality into your evening prayer?


Describe your craving or ask a question (or type 'quit' to exit):  ok cool  by then im going to do that


I'm really glad to hear you're planning on doing a calming, at-home yoga session, Sarah. It's a beautiful opportunity to connect with yourself and your Higher Power, bringing in moments of peace that can help shift your focus away from cravings. As you prepare for your evening, how might you incorporate feelings of love and spirituality into your practice to strengthen your resolve and nurture your soul?


Describe your craving or ask a question (or type 'quit' to exit):  quit


Ending conversation. Take care!


## 10. Extract Conversation Insights

Use the LLM to extract structured insights from the conversation history.

In [171]:
# --- Conversation Insights Extraction ---
import re
def extract_conversation_insights(conversation_history):
    """Extract insights from the conversation using LLM and return as a dict."""
    conversation_text = ""
    for msg in conversation_history:
        role = "Sponsor" if msg.get("role") == "sponsor" else "User"
        conversation_text += f"{role}: {msg['content']}\n"

    extraction_prompt = f"""Based on this initial MA conversation, extract the following information:

{conversation_text}

Extract and format as JSON:
{{
    "user_name": "their first name if mentioned, or null",
    "spiritual_view": "christian/atheist/agnostic/spiritual/unknown",
    "openness_level": "low/medium/high/unknown",
    "step_content": "any step they mentioned or null",
    "persona_used": "spiritual_explorer",
    "emotional_tone_start": "nervous/hopeful/resistant/curious/unknown", 
    "emotional_tone_end": "more_open/same/more_resistant/hopeful/unknown",
    "key_insights": ["insight1", "insight2", "etc"],
    "action_items": ["what they agreed to do", "etc"],
    "ready_for_steps": true/false,
    "conversation_summary": "brief 1-2 sentence summary"
}}

Only include information explicitly mentioned. Use "unknown" for unclear items."""

    response = openai_client.chat.completions.create(
        model="gpt-4o",
        messages=[{"role": "user", "content": extraction_prompt}],
        temperature=0.1
    )
    response_text = response.choices[0].message.content.strip()
    # Remove code fences if present
    response_text = re.sub(r"^```(?:json)?\s*|```$", "", response_text, flags=re.MULTILINE).strip()
    try:
        return json.loads(response_text)
    except Exception as e:
        print("Failed to parse LLM output as JSON:", e)
        print("LLM output was:", response.choices[0].message.content)
        return {}

In [172]:
extracted_insights = extract_conversation_insights(current_conversation_history)

## 11. Build and Save Conversation Record

Create a structured record of the conversation and save it to `conversations.json`.

In [173]:
# --- Build Conversation Record ---
import uuid
from datetime import datetime

def build_conversation_record(conversation_history, insights, persona_used="spiritual_explorer", conversation_type="initial_meeting"):
    """Build a detailed conversation record for saving."""
    total_messages = len(conversation_history)
    duration_estimate = total_messages * 2  # e.g., 2 min per message

    conversation_record = {
        "conversation_id": str(uuid.uuid4())[:8],
        "user_id": None,  # To be filled in later
        "date": datetime.now().strftime("%Y-%m-%d"),
        "timestamp": datetime.now().isoformat(),
        "step_content": insights.get("step_content"),
        "persona_used": persona_used,
        "conversation_type": conversation_type,
        "user_name": insights.get("user_name"),
        "spiritual_view": insights.get("spiritual_view"),
        "openness_level": insights.get("openness_level"),
        "emotional_tone_start": insights.get("emotional_tone_start", "unknown"),
        "emotional_tone_end": insights.get("emotional_tone_end", "unknown"),
        "total_messages": total_messages,
        "duration_minutes": duration_estimate,
        "key_insights": insights.get("key_insights", []),
        "action_items": insights.get("action_items", []),
        "ready_for_steps": insights.get("ready_for_steps", False),
        "follow_up_suggested": insights.get("ready_for_steps", False),
        "rag_sources_used": [],  # Add if you track RAG sources
        "conversation_data": {
            "dialogue": conversation_history,
            "extracted_insights": insights,
            "summary": insights.get("conversation_summary", "Initial MA conversation")
        }
    }
    return conversation_record

In [174]:
conversation_record = build_conversation_record(current_conversation_history, extracted_insights, persona_used=persona, conversation_type="cravings_support")

In [175]:
# --- Save to JSON files ---
from pathlib import Path

users_path = Path("/workspaces/llm-zoomcamp/mai/context/users/users.json")
conversations_path = Path("/workspaces/llm-zoomcamp/mai/context/conversations/conversations.json")
spiritual_profile_path = Path("/workspaces/llm-zoomcamp/mai/context/spiritual_profile/spiritual_profile.json")

In [176]:
# Load existing conversations
if conversations_path.exists():
    with open(conversations_path, "r") as f:
        conversations = json.load(f)
else:
    conversations = []

# Add the new record
conversations.append(conversation_record)

# Save back
with open(conversations_path, "w") as f:
    json.dump(conversations, f, indent=2)

## 12. Update Spiritual Profile

Archive the previous profile state and update the current profile with new persona and insights.

In [178]:
if spiritual_profile_path.exists():
    with open(spiritual_profile_path, "r") as f:
        profiles = json.load(f)
    for profile in profiles:
        if profile["user_id"] == user_id:
            # Archive the current profile to history with timestamp
            if "history" not in profile:
                profile["history"] = []
            profile["history"].append({
                **profile["current_profile"],
                "archived_at": datetime.now().isoformat()
            })
            # Update current_profile with persona used in this session
            profile["current_profile"]["persona"] = persona  # <-- use the variable, not a string
            profile["current_profile"]["updated_at"] = datetime.now().isoformat()
            # Optionally update openness_level, etc. from extracted_insights
            if extracted_insights.get("openness_level") and extracted_insights["openness_level"] != "unknown":
                profile["current_profile"]["openness_level"] = extracted_insights["openness_level"]
    with open(spiritual_profile_path, "w") as f:
        json.dump(profiles, f, indent=2)

## 13. Update Users Table

Update the user's persona, openness level, and step progress in `users.json`.

In [179]:
from datetime import datetime

if users_path.exists():
    with open(users_path, "r") as f:
        users = json.load(f)
    for user in users:
        if user["user_id"] == user_id:
            # Update persona to the one used in this session
            user["persona"] = persona
            # Optionally update other fields from extracted_insights
            if extracted_insights.get("openness_level") and extracted_insights["openness_level"] != "unknown":
                user["openness_level"] = extracted_insights["openness_level"]
            if extracted_insights.get("step_content") and extracted_insights["step_content"] != "unknown":
                user["step_in_progress"] = extracted_insights["step_content"]
            # Always update last_updated timestamp
            user["last_updated"] = datetime.now().isoformat()
    with open(users_path, "w") as f:
        json.dump(users, f, indent=2)

## 14. Reset Conversation History

Clear the conversation history to prepare for a new session.

In [180]:
# ...after all saving/updating code - reitinitalize current conversation history
current_conversation_history = []


In [181]:
conversation_record

{'conversation_id': '2624ba7e',
 'user_id': None,
 'date': '2025-07-02',
 'timestamp': '2025-07-02T20:18:47.368154',
 'step_content': None,
 'persona_used': 'steady_believer',
 'conversation_type': 'cravings_support',
 'user_name': 'Sarah',
 'spiritual_view': 'unknown',
 'openness_level': 'high',
 'emotional_tone_start': 'nervous',
 'emotional_tone_end': 'more_open',
 'total_messages': 20,
 'duration_minutes': 40,
 'key_insights': ['Sarah experiences strong cravings for marijuana.',
  'She finds gratitude and spiritual practices helpful in managing cravings.',
  'She is considering yoga as a form of self-care and spiritual practice.'],
 'action_items': ['Do yoga with Adriene on YouTube',
  'Order food in',
  'Create a calming atmosphere at home',
  'Pray after yoga when feeling calm'],
 'ready_for_steps': False,
 'follow_up_suggested': False,
 'rag_sources_used': [],
 'conversation_data': {'dialogue': [{'role': 'user',
    'content': 'i cant stop thinking about weed'},
   {'role': 'spo